In [ ]:
import telebot
from telebot import types
import fasttext
import fasttext.util
fasttext.util.download_model('ru', if_exists='ignore')  # English
import random
from numpy import dot
from numpy.linalg import norm

ft = fasttext.load_model('cc.ru.300.bin')

In [ ]:
class TestBot:
    def __init__(self, token=None, ft=None):
        self.bot = telebot.TeleBot('6505268192:AAFk3XrQjkDHvSziBe2Wq5OLLqAxLnIF7bg')
        self.words = "краска, разработка, судья, выпуск, старуха, механизм, организм, вариант, опыт, действительность, инженер, деньги, дом, ожидание, произведение, карта, открытие, полоса, море, поддержка, здоровье, TRUE, ладонь, река, колено, ощущение, житель, правда, чемпионат, свойство, значение, отличие, площадь, гостиница, государство, апрель, телефон, президент, федерация, факт, выборы, девочка, занятие, вход, представление, секретарь, сеть, законодательство, автомат, господь, чемпионат, партнер, пенсия, ветер, постель, источник, кредит, запад, смех, способность, песок, край, сбор, мама, январь, эксперимент, хозяйка, суть, поэзия, федерация, ужас, пол, урок, чтение, тайна, формирование, масштаб, суд, лоб, дождь, шаг, общество, разработка, мужик, стиль, покупатель, неделя, ребенок, вывод, обучение, август, продажа, зуб, соглашение, увеличение, девушка, потеря, господь, родственник, редакция, утро, стул, увеличение, стул, лес, сердце, храм, помещение, дума, двигатель, народ, диван, нос, глаз, постановление, часы, хозяйка, разница, карта, писатель, частность, исследование, объект, золото, увеличение, реализация, TRUE, оценка, усилие, газета, отказ, глубина, помощник, причина, удар, оплата, линия, сестра, октябрь, ресторан, состояние, спина, пример, ребенок, доказательство, газ, главное, статья, двор, мальчик, ящик, уровень, создание, девочка, развитие, выполнение, старуха, комплекс, обращение, ребята, версия, перевод, реформа, кредит, солдат, программа, средство, личность, постель, гора, промышленность, продукция, давление, рыба, ожидание, храм, скорость, требование, сотня, январь, фонд, коридор, понимание, заместитель, река, оружие, эффект, билет, поэт, ракета, язык, вода, пакет, достижение, солдат, рыба, контакт, рабочий, выборы, обучение"
        self.words = [x.strip() for x in self.words.split(', ')]
        self.ft = ft

    
        

    def start_test(self, message):
        self.bot_word = random.choice(self.words)
        print(self.bot_word, "bot_word")
        self.word_vector = self.ft.get_word_vector(self.bot_word)
        self.bot.send_message(message.chat.id, "Слово сгенеррованно, начинайте гадать")

        self.answers = []
        self.user_words = set()

    def get_text_messages(self, message):
        print(message.text)
        user_word = message.text.lower()
        if user_word == self.bot_word:
            self.bot.send_message(message.chat.id, "Поздравляю, Вы угадали слово! Начните новую игру через \start")
        else:
            user_embed = self.ft.get_word_vector(message.text)
            cos_sim = dot(user_embed, self.word_vector)/(norm(user_embed)*norm(self.word_vector))
            progress_bar = self.printProgressBar((cos_sim*10000)//100, total=100)
            if user_word not in self.user_words:
                self.user_words.add(user_word)
                self.answers.append(((cos_sim*10000)//100, user_word, progress_bar))
                self.bot.send_message(message.chat.id, user_word + ' ' + progress_bar)
            self.answers = sorted(self.answers)[-5:]
            res_str = ""
            for i in self.answers:
                s = str(i[1]) + ' ' + i[2] + '\n'
                res_str += s
            self.bot.send_message(message.chat.id, res_str)
            
    def printProgressBar(self, iteration, total, prefix = '', suffix = '', decimals = 1, length = 30, fill = '█', printEnd = "\r"):
        percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
        filledLength = int(length * iteration // total)
        bar = fill * filledLength + '-' * (length - filledLength)
        return f'\r{prefix} |{bar}| {percent}% {suffix}'
        


test_bot = TestBot(ft=ft) #bot_token)

@test_bot.bot.message_handler(commands=['start'])
def handle_start(message):
    test_bot.start_test(message)

@test_bot.bot.message_handler(content_types=['text'])
def handle_message(message):
    test_bot.get_text_messages(message)

print("DONE")

In [ ]:
import time
while True:
    try:
        test_bot.bot.polling(none_stop=True)

    except Exception as e:
        print(e)  # или просто print(e) если у вас логгера нет,
        # или import traceback; traceback.print_exc() для печати полной инфы
        time.sleep(15)

In [ ]:
test_bot.answers[0]

In [ ]:
sorted([3,4,2])[1:]